En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
# El filepath corresponde a la ubicación en BigQuery
PROJECT_ID: str = "de-challenge-gm"
DATASET_ID: str = "tweets"
TABLE_NAME: str = "farmers-protest-tweets"

file_path = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}"
print(file_path)

de-challenge-gm.tweets.farmers-protest-tweets


In [2]:
# Ejecución q1_memory
import q1_memory
result = q1_memory.q1_memory(file_path)
print(result)

2024-07-05 14:17:06,604 - INFO - Starting: q1_memory
2024-07-05 14:17:06,605 - INFO - Ejecutando query
2024-07-05 14:17:08,556 - INFO - Tiempo de ejecución de la query (lado del cliente, python, hasta recibir la respuesta): 1.95 segundos
2024-07-05 14:17:08,561 - INFO - Se uso query cache?: False
2024-07-05 14:17:08,562 - INFO - Bigquery Job Detail:
2024-07-05 14:17:08,562 - INFO - Job ID: 435de2c8-bcbd-440f-ae79-c73f9cacf752
2024-07-05 14:17:08,563 - INFO - Job Status: DONE
2024-07-05 14:17:08,563 - INFO - Tiempo inicio en máquina GCP: 2024-07-05 18:17:07.289000+00:00
2024-07-05 14:17:08,564 - INFO - Tiempo fin en máquina GCP: 2024-07-05 18:17:08.235000+00:00
2024-07-05 14:17:08,565 - INFO - Tiempo de ejecución total en servidor GCP: 0.946 segundos
2024-07-05 14:17:08,565 - INFO - Tiempo de ejecución total en cliente (python): 1.95 segundos
2024-07-05 14:17:08,566 - INFO - Delta de tiempo (costo de red, serialización, SO, etc.): 1.00 segundos
2024-07-05 14:17:08,566 - INFO - Bytes pro

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


In [3]:
# Ejecución q1_time
import q1_time
result = q1_time.q1_time(file_path)
print(result)

2024-07-05 14:17:17,526 - INFO - Starting: q1_time
2024-07-05 14:17:17,526 - INFO - Ejecutando query
2024-07-05 14:17:19,151 - INFO - Tiempo de ejecución de la query (lado del cliente, python, hasta recibir la respuesta): 1.62 segundos
2024-07-05 14:17:19,152 - INFO - Se uso query cache?: False
2024-07-05 14:17:19,153 - INFO - Bigquery Job Detail:
2024-07-05 14:17:19,153 - INFO - Job ID: 80b46508-6a65-4ee4-9ecb-63c551d06751
2024-07-05 14:17:19,154 - INFO - Job Status: DONE
2024-07-05 14:17:19,154 - INFO - Tiempo inicio en máquina GCP: 2024-07-05 18:17:18.112000+00:00
2024-07-05 14:17:19,155 - INFO - Tiempo fin en máquina GCP: 2024-07-05 18:17:18.789000+00:00
2024-07-05 14:17:19,155 - INFO - Tiempo de ejecución total en servidor GCP: 0.677 segundos
2024-07-05 14:17:19,156 - INFO - Tiempo de ejecución total en cliente (python): 1.62 segundos
2024-07-05 14:17:19,157 - INFO - Delta de tiempo (costo de red, serialización, SO, etc.): 0.95 segundos
2024-07-05 14:17:19,157 - INFO - Bytes proce

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


In [4]:
# Ejecución q2_memory
import q2_memory
result = q2_memory.q2_memory(file_path)
print(result)

2024-07-05 14:25:33,351 - INFO - Starting: q2_memory
2024-07-05 14:25:33,352 - INFO - Ejecutando query
2024-07-05 14:25:35,493 - INFO - Tiempo de ejecución de la query (lado del cliente, python, hasta recibir la respuesta): 2.14 segundos
2024-07-05 14:25:35,494 - INFO - Se uso query cache?: False
2024-07-05 14:25:35,495 - INFO - Bigquery Job Detail:
2024-07-05 14:25:35,496 - INFO - Job ID: 75816922-a221-4035-90bf-1cbcac488abb
2024-07-05 14:25:35,496 - INFO - Job Status: DONE
2024-07-05 14:25:35,496 - INFO - Tiempo inicio en máquina GCP: 2024-07-05 18:25:34.067000+00:00
2024-07-05 14:25:35,498 - INFO - Tiempo fin en máquina GCP: 2024-07-05 18:25:35.148000+00:00
2024-07-05 14:25:35,498 - INFO - Tiempo de ejecución total en servidor GCP: 1.081 segundos
2024-07-05 14:25:35,499 - INFO - Tiempo de ejecución total en cliente (python): 2.14 segundos
2024-07-05 14:25:35,499 - INFO - Delta de tiempo (costo de red, serialización, SO, etc.): 1.06 segundos
2024-07-05 14:25:35,500 - INFO - Bytes pro

[('✊', 1724), ('❤', 1471), ('\u200d', 418), ('✌', 202), ('☮', 170), ('♂', 139), ('♀', 113), ('✍', 91), ('♥', 56), ('⚔', 48)]


In [5]:
# Ejecución q2_time
import q2_time
result = q2_time.q2_time(file_path)
print(result)

2024-07-05 14:25:49,171 - INFO - Starting: q2_time
2024-07-05 14:25:49,172 - INFO - Ejecutando query
2024-07-05 14:25:50,585 - INFO - Tiempo de ejecución de la query (lado del cliente, python, hasta recibir la respuesta): 1.41 segundos
2024-07-05 14:25:50,586 - INFO - Se uso query cache?: False
2024-07-05 14:25:50,587 - INFO - Bigquery Job Detail:
2024-07-05 14:25:50,587 - INFO - Job ID: fead4f9c-efc8-4ffd-badf-c1027eceae8c
2024-07-05 14:25:50,589 - INFO - Job Status: DONE
2024-07-05 14:25:50,590 - INFO - Tiempo inicio en máquina GCP: 2024-07-05 18:25:49.753000+00:00
2024-07-05 14:25:50,591 - INFO - Tiempo fin en máquina GCP: 2024-07-05 18:25:50.137000+00:00
2024-07-05 14:25:50,591 - INFO - Tiempo de ejecución total en servidor GCP: 0.384 segundos
2024-07-05 14:25:50,592 - INFO - Tiempo de ejecución total en cliente (python): 1.41 segundos
2024-07-05 14:25:50,592 - INFO - Delta de tiempo (costo de red, serialización, SO, etc.): 1.03 segundos
2024-07-05 14:25:50,593 - INFO - Bytes proce

[('✊', 1724), ('❤', 1471), ('\u200d', 418), ('✌', 202), ('☮', 170), ('♂', 139), ('♀', 113), ('✍', 91), ('♥', 56), ('⚔', 48)]


In [ ]:
# Ejecución q3_memory (TODO)
import q3_memory
result = q3_memory.q3_memory(file_path)
print(result)

In [ ]:
# Ejecución q3_time (TODO)
import q3_time
result = q3_time.q3_time(file_path)
print(result)

# Q1: top 10 fechas donde hay más tweets

## Tabla Comparativa de Mediciones

### Configuración del job

| Parámetros                                      | q1_memory | q1_time |
|------------------------------------------------|-----------|---------|
| QUERY_CACHE                                    | False     | False   |
| QUERY_PRIORITY                                 | BATCH     | INTERACTIVE|

### Resultados mediciones

| Medicion                                      | q1_memory | q1_time |
|-----------------------------------------------|-----------|---------|
| Tiempo ejecución en cliente                   | 1.95 s    | 1.62 s  |
| Tiempo ejecución en GCP (server)              | 0.946 s   | 0.677 s |
| Delta tiempo (client-server)                  | 1.00 s    | 0.95 s  |
| Memoria usada por cliente python              | NA        | NA      |
| CPU usado por cliente python                  | NA        | NA      |
| Bytes Procesados en GCP (server)              | 2.45 MB   | 2.45 MB |
| Bytes Facturados en GCP (server)              | 10.49 MB  | 10.49 MB|
| Slot Machine tiempo usado (GCP CPU Time) (ms) | 23348 ms  | 1890 ms |
| BigQuery Total steps en QueryPlan             | 11        | 5       |

### Resultados Query Plan

#### Query Plan: q1_memory

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 13              | COMPLETE |
| S01   | Sort+                | 13           | 10              | COMPLETE |
| S02   | Sort                 | 10           | 10              | COMPLETE |
| S03   | Coalesce             | 10           | 10              | COMPLETE |
| S04   | Join+                | 117537       | 44159           | COMPLETE |
| S06   | Sort                 | 10           | 10              | COMPLETE |
| S07   | Coalesce             | 10           | 10              | COMPLETE |
| S08   | Join+                | 44169        | 10              | COMPLETE |
| S09   | Aggregate+           | 10           | 10              | COMPLETE |
| S0A   | Output               | 10           | 10              | COMPLETE |

#### Query Plan: q1_time

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 51646           | COMPLETE |
| S01   | Aggregate+           | 51646        | 13              | COMPLETE |
| S02   | Sort+                | 13           | 10              | COMPLETE |
| S03   | Sort+                | 10           | 10              | COMPLETE |
| S04   | Output               | 10           | 10              | COMPLETE |

### Comparación Query Plan enfoque memory vs time

| Step | q1_memory (Q1_MEMORY_QUERY)                                   | q1_time (Q1_TIME_QUERY)                                    |
|------|---------------------------------------------------------------|-----------------------------------------------------------|
| S00  | **Input**: Records read: 117407, Records written: 13, Status: COMPLETE | **Input**: Records read: 117407, Records written: 51646, Status: COMPLETE |
| S01  | **Sort+**: Records read: 13, Records written: 10, Status: COMPLETE | **Aggregate+**: Records read: 51646, Records written: 13, Status: COMPLETE |
| S02  | **Sort**: Records read: 10, Records written: 10, Status: COMPLETE | **Sort+**: Records read: 13, Records written: 10, Status: COMPLETE |
| S03  | **Coalesce**: Records read: 10, Records written: 10, Status: COMPLETE | **Sort+**: Records read: 10, Records written: 10, Status: COMPLETE |
| S04  | **Join+**: Records read: 117537, Records written: 44159, Status: COMPLETE | **Output**: Records read: 10, Records written: 10, Status: COMPLETE |
| S05  | **Aggregate**: Records read: 44159, Records written: 44159, Status: COMPLETE |                                                           |
| S07  | **Coalesce**: Records read: 44159, Records written: 44159, Status: COMPLETE |                                                           |
| S08  | **Join+**: Records read: 44169, Records written: 10, Status: COMPLETE |                                                           |
| S09  | **Aggregate+**: Records read: 10, Records written: 10, Status: COMPLETE |                                                           |
| S0A  | **Output**: Records read: 10, Records written: 10, Status: COMPLETE |                                                           |

# Q2: top 10 emojis más usados con su respectivo conteo

## Tabla Comparativa de Mediciones

### Configuración del job

| Parámetros     | q2_memory | q2_time |
|----------------|-----------|---------|
| QUERY_CACHE    | False     | False   |
| QUERY_PRIORITY | INTERACTIVE     | INTERACTIVE |

### Resultados mediciones

| Medicion                                      | q2_memory | q2_time |
|-----------------------------------------------|-----------|---------|
| Tiempo ejecución en cliente                   | 2.14 s    | 1.41 s  |
| Tiempo ejecución en GCP (server)              | 1.081 s   | 0.384 s |
| Delta tiempo (client-server)                  | 1.06 s    | 1.03 s  |
| Memoria usada por cliente python              | NA        | NA      |
| CPU usado por cliente python                  | NA        | NA      |
| Bytes Procesados en GCP (server)              | 20.86 MB  | 20.86 MB|
| Bytes Facturados en GCP (server)              | 22.02 MB  | 22.02 MB|
| Slot Machine tiempo usado (GCP CPU Time) (ms) | 31581 ms  | 787 ms  |
| BigQuery Total steps en QueryPlan             | 11        | 3       |

### Resultados Query Plan

#### Query Plan: q2_memory

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 419             | COMPLETE |
| S01   | Aggregate            | 419          | 85              | COMPLETE |
| S02   | Aggregate+           | 85           | 1               | COMPLETE |
| S03   | Aggregate            | 1            | 1               | COMPLETE |
| S04   | Sort+                | 86           | 85              | COMPLETE |
| S05   | Aggregate            | 85           | 82              | COMPLETE |
| S07   | Compute              | 85           | 85              | COMPLETE |
| S08   | Coalesce             | 85           | 85              | COMPLETE |
| S09   | Join+                | 932          | 10              | COMPLETE |
| S0A   | Output               | 10           | 10              | COMPLETE |

#### Query Plan: q2_time

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 419             | COMPLETE |
| S01   | Sort+                | 419          | 10              | COMPLETE |
| S02   | Output               | 10           | 10              | COMPLETE |

### Comparación Query Plan enfoque memory vs time

| Step | q2_memory (Q2_MEMORY_QUERY)                                   | q2_time (Q2_TIME_QUERY)                                    |
|------|---------------------------------------------------------------|-----------------------------------------------------------|
| S00  | **Input**: Records read: 117407, Records written: 419, Status: COMPLETE | **Input**: Records read: 117407, Records written: 419, Status: COMPLETE |
| S01  | **Aggregate**: Records read: 419, Records written: 85, Status: COMPLETE | **Sort+**: Records read: 419, Records written: 10, Status: COMPLETE |
| S02  | **Aggregate+**: Records read: 85, Records written: 1, Status: COMPLETE | **Output**: Records read: 10, Records written: 10, Status: COMPLETE |
| S03  | **Aggregate**: Records read: 1, Records written: 1, Status: COMPLETE |                                                           |
| S04  | **Sort+**: Records read: 86, Records written: 85, Status: COMPLETE |                                                           |
| S05  | **Aggregate**: Records read: 85, Records written: 82, Status: COMPLETE |                                                           |
| S07  | **Compute**: Records read: 85, Records written: 85, Status: COMPLETE |                                                           |
| S08  | **Coalesce**: Records read: 85, Records written: 85, Status: COMPLETE |                                                           |
| S09  | **Join+**: Records read: 932, Records written: 10, Status: COMPLETE |                                                           |
| S0A  | **Output**: Records read: 10, Records written: 10, Status: COMPLETE |                                                           |

# Q3: top 10 histórico de usuarios (username) más influyentes

## Tabla Comparativa de Mediciones

### Configuración del job

| Parámetros                                      | q3_memory | q3_time |
|-------------------------------------------------|-----------|---------|
| QUERY_CACHE                                     | False     | False   |
| QUERY_PRIORITY                                  | BATCH     | PRIORITY|

### Resultados mediciones

| Medicion                                      | q3_memory | q3_time |
|-----------------------------------------------|-----------|---------|
| Tiempo ejecución en cliente                   | 2.33 s    | 1.48 s  |
| Tiempo ejecución en GCP (server)              | 0.48 s    | 0.515 s |
| Delta tiempo (client-server)                  | 1.85 s    | 0.97 s  |
| Memoria usada por cliente python                     | NA   | NA |
| CPU usado por cliente python                     | NA   | NA |
| Bytes Procesados en GCP (server)              | 7.16 MB   | 1.27 MB |
| Bytes Facturados en GCP (server)              | 10.49 MB  | 10.49 MB|
| Slot Machine tiempo usado (GCP CPU Time) (ms) | 616 ms    | 681 ms  |
| BigQuery Total steps en QueryPlan             | 3         | 3       |

### Resultados Query Plan

#### Query Plan: q3_memory

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 31260           | COMPLETE |
| S01   | Sort+                | 31260        | 10              | COMPLETE |
| S02   | Output               | 10           | 10              | COMPLETE |

#### Query Plan: q3_time

| Step  | Descripción         | Records Read | Records Written | Status   |
|-------|----------------------|--------------|-----------------|----------|
| S00   | Input                | 117407       | 31260           | COMPLETE |
| S01   | Sort+                | 31260        | 10              | COMPLETE |
| S02   | Output               | 10           | 10              | COMPLETE |

### Comparación Query Plan enfoque memory vs time

| Step | q3_memory (Q3_MEMORY_QUERY)                                   | q3_time (Q3_TIME_QUERY)                                    |
|------|---------------------------------------------------------------|-----------------------------------------------------------|
| S00  | **Input**: Records read: 117407, Records written: 31260, Status: COMPLETE | **Input**: Records read: 117407, Records written: 31260, Status: COMPLETE |
| S01  | **Sort+**: Records read: 31260, Records written: 10, Status: COMPLETE | **Sort+**: Records read: 31260, Records written: 10, Status: COMPLETE |
| S02  | **Output**: Records read: 10, Records written: 10, Status: COMPLETE | **Output**: Records read: 10, Records written: 10, Status: COMPLETE |